# Assign integer index to H3 cells using group_by for node assignment

In [1]:
# for compatability.....
# !pip install polars, rustworkx, pytorch, feature-engineering-polars==0.2.0


import os
import polars as pl
import pandas as pd

# import cugraph
import rustworkx as rx
import cdlib
import networkx as nx

import pyvis as pv


from datetime import datetime, timedelta

In [2]:
data_folder = '/mnt/c/Users/Asus/Code/data/AIS/enriched_AIS'
data = [file for file in os.listdir(data_folder) if 'parquet' in file]

# Communities for a geospatial result | edges are MMSI<->H3_cell
### results should be communities of vessls with similar destinations/routs, or whatever it is boats do

In [3]:
# THIS is how we will assign H3 indexes
# just remember to delete at the end, otherwise better off with a hash table or something
# polars for ETL: be sure to do lazy or streaming for query optimization
ais =  (
    pl.concat(
        [dat for dat in 
         [pl.scan_parquet(f"""{data_folder}{os.sep}{file}""") 
          for file in data 
          if 'parquet' in file] 
         if 'LON' in dat.columns]
    , how='diagonal'
    )
    .with_columns(
        pl.col('BaseDateTime').str.strptime(pl.Datetime, format='%Y-%m-%dT%H:%M:%S'))
    .unique()
)

INDEX_OFFSET = len(ais.select(pl.col('H3')).unique().collect())
# now get our edges (and nodes), by grouping on MMSI and H3 cells
ais = (
    ais
    .group_by(pl.col('H3'))
    .agg(pl.all())
    .with_row_index(name='H3_index')
    .explode(pl.exclude(['H3','H3_index']))
    .group_by(pl.col('MMSI'))
    .agg(pl.all())    
    .with_row_index(
        name='MMSI_index', 
        offset=INDEX_OFFSET)
    .explode(pl.exclude(['MMSI','MMSI_index']))
    .group_by(pl.col(['H3_index', 'MMSI_index']))
    .agg(pl.all())
    .explode(pl.exclude(['H3_index', 'MMSI_index']))
    .with_columns(
        pl.struct(pl.col(['H3_index', 'MMSI_index'])).alias('edge'))
    .with_columns(
        pl.col(['H3_index', 'MMSI_index']).cast(pl.Int64))
    .select(
        pl.col(['H3','MMSI', 'H3_index','MMSI_index', 'LON','LAT', 'BaseDateTime']))
).collect()


# farts, lat and long are flipped in the ETL somehow.....
ais = ais.with_columns(pl.col('LON').alias('latitude')).with_columns(pl.col('LAT').alias('longitude'))
ais = ais.with_columns(pl.col('longitude').alias('LON')).with_columns(pl.col('latitude').alias('LAT'))

edge_table = ais.select(pl.col(['H3', 'MMSI', 'BaseDateTime'])).group_by(pl.col(['H3','MMSI'])).len()

In [4]:
wpi = pl.read_csv('/mnt/d/data/Ports/major_ports.csv').drop('OBJECTID')

In [5]:
import requests

password = 'wdi85v#MvBJZ4#M'
account = 'nightwatch'

# xml return
# url = f"http://api.geonames.org/findNearbyPlaceName?lat=47.3&lng=9&username={account}"

def get_PlaceName(
    latitude,
    longitude,
    verbose=False
):
    
    # url = f"http://api.geonames.org/findNearbyPlaceNameJSON?lat={latitude}&lng={longitude}&username={account}"
    url = f"""http://api.geonames.org/findNearbyPlaceNameJSON?lat={latitude}&lng={longitude}&radius=300&username={account}"""
    response = requests.get(url)

    if verbose==True:
        print(type(latitude), latitude)
        print(url)
        
    if response.status_code == 200:
        return response.json()
    
    else:
        return response.text


In [6]:
# limit the query to geonames by binning in H3 cells
H3_MASK = 68

ais = (
    ais.with_columns(
        pl.col('H3').str.slice(0, H3_MASK)
        .alias('H3_MASK')))

coordinates = (
    ais.group_by(pl.col('H3_MASK'))
    .agg(pl.col(['LON', 'LAT']).mean())
)
len(coordinates)

46243

In [12]:
places = (
    coordinates
    .with_columns(
        pl.struct(
            pl.col(['LON','LAT']))
        .map_elements(
            lambda xx: get_PlaceName(xx['LON'], xx['LAT']))
    .alias('response'))
    # .unnest('response')
    # .explode(pl.col('geonames'))
    # .unnest('geonames')
    # .unnest('adminCodes1')
)
places

H3_MASK,LON,LAT,response
str,f64,f64,str
"""872913204fffff…",34.670242,-121.163035,"""{""status"":{""me…"
"""872819a66fffff…",43.115434,-125.735206,"""{""status"":{""me…"
"""872922412fffff…",30.358475,-123.4595,"""{""status"":{""me…"
"""87291a208fffff…",36.32935,-122.476929,"""{""status"":{""me…"
"""87281d3b6fffff…",42.387796,-125.063012,"""{""status"":{""me…"
…,…,…,…
"""8729181aafffff…",35.28424,-123.0902,null
"""8728f1a1efffff…",45.839067,-124.496667,null
"""87280440dfffff…",39.220116,-124.233712,null


In [ ]:
# ais.join(places, how='left', on='H3_MASK')
places = places.unnest('response').explode(pl.col('geonames')).unnest('geonames').unnest('adminCodes1')

PLACES_FILEPATH = '/mnt/d/data/Ports/ais_places.parquet'
places.write_parquet(PLACES_FILEPATH)

In [13]:
from vincenty import vincenty

In [ ]:
# ais_temp = ais.group_by('H3_MASK').agg(pl.all())
# places_temp = places.drop_nulls().group_by('H3_MASK').agg(pl.all())

# temp = {}
# for H3 in ais['H3_MASK'].unique():
#     AT = ais_temp.filter(pl.col('H3_MASK') == H3)
#     PT = places_temp.filter(pl.col('H3_MASK') == H3)
#     if len(AT) > 0 and len(PT) >1:
#         temp[H3] = AT.join(PT, how='left', on='H3_MASK')

In [ ]:
# places.filter(pl.col('lng').is_not_null() & pl.col('lat').is_not_null()).group_by('H3_MASK').agg(pl.all())
places.drop_nulls()

In [ ]:
import networkx as nx
import rustworkx as rx
from cdlib import algorithms, evaluation, viz

import hvplot.networkx as hvnx

from pyvis.network import Network
from pyvis import network as net

# NetworkX

In [ ]:
nx_G = nx.Graph()

for e in edge_table.to_numpy():
    nx_G.add_edge(e[0]['H3_index'], e[0]['MMSI_index'], weight=e[1])

# different algorithms
louvain_coms = algorithms.louvain(nx_G)

lp_coms = algorithms.label_propagation(nx_G)

evaluation.normalized_mutual_information(louvain_coms, lp_coms)

# too big to plot
# viz.plot_community_graph(
#     nx_G, # graph – NetworkX/igraph graph
#     louvain_coms, # partition – NodeClustering object
#     figsize=(15.0,15.0), #– the figure size; it is a pair of float, default (8, 8)
#     node_size=200, #– int, default 200
#     plot_overlaps=False, #– bool, default False. Flag to control if multiple algorithms memberships are plotted.
#     plot_labels=True, #– bool, default False. Flag to control if node labels are plotted.
#     cmap=None, #– str or Matplotlib colormap, Colormap(Matplotlib colormap) for mapping intensities of nodes. If set to None, original colormap is used..
#     top_k=10, #– int, Show the top K influential communities. If set to zero or negative value indicates all.
#     min_size=10, #– int, Exclude communities below the specified minimum size
# )

In [ ]:
# alright, so lets look at these ourselves....
communities = {
    int(f"{len(com)}{i}"):com for i, com in enumerate(louvain_coms.communities)
}

biggest = communities[max(communities.keys())]

biggest = ais.filter(
    pl.col('H3_index').is_in(biggest)
    | pl.col('MMSI_index').is_in(biggest)
).groupby(pl.col('edge')).agg(pl.all())

In [ ]:

    
def pl_to_nx(
    frame
):
    frame = frame.select(pl.col(['H3_index', 'MMSI_index', 'BaseDateTime'])).groupby(pl.col(['H3_index','MMSI_index'])).count()
    graph = nx_G = nx.Graph()
    
    for e in frame.to_numpy():
        graph.add_edge(
            e[0], 
            e[0], 
            weight=e[1]
        )
    
    return graph

b_nx_G = pl_to_nx(ais)
hvnx.draw(b_nx_G, with_labels=True)

In [ ]:
edge_table.groupby('MMSI').agg(pl.all())

In [ ]:
from pyvis.network import Network
import pandas as pd

got_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

# set the physics layout of the network
got_net.barnes_hut()

sources = edge_table['MMSI']
targets = edge_table['H3']
weights = edge_table['count']

edge_data = zip(sources, targets, weights)

for e in edge_data:
                src = e[0]
                dst = e[1]
                w = e[2]
                got_net.add_node(src, src, title=src)
                got_net.add_node(dst, dst, title=dst)
                got_net.add_edge(src, dst, value=w)

neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
for node in got_net.nodes:
                node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
                node["value"] = len(neighbor_map[node["id"]])

got_net.show("ais.html")

In [ ]:
# algorithm eval metrics
# louvain_coms.normalized_mutual_information(lp_coms)

# louvain_coms.average_internal_degree()
# evaluation.average_internal_degree(nx_G, louvain_coms)


In [ ]:
# community labels
# lp_coms.communities

# these are the community assignments per node
# louvain_coms.to_node_community_map()

In [ ]:
# plot with pyvis
pos = nx.spring_layout(nx_G)
# viz.plot_network_clusters(nx_G, louvain_coms, pos)



# viz.plot_network_clusters(nx_G, louvain_coms, pos, figsize=(15, 15))

In [ ]:
# plot with hvplot/bokeh

hvnx.draw(nx_G, with_labels=True)

In [ ]:
net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

net.add_nodes(
    [1,2,3], 
    value=[10, 100, 400],
    title=['I am node 1', 'node 2 here', 'and im node 3'],
    x=[21.4, 54.2, 11.2],
    y=[100.2, 23.54, 32.1],
    label=['NODE 1', 'NODE 2', 'NODE 3'],
    color=['#00ff1e', '#162347', '#dd4b39']
)

net.add_edge(0, 1, weight=.87)
net.toggle_physics(True)

In [ ]:
net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

# set the physics layout of the network
net.barnes_hut()

source_label = geo['H3']
sources = geo['H3_index']
target_label = geo['MMSI']
targets = geo['MMSI_index']
weights = geo['count']

edge_data = list(zip(sources, source_label, targets, target_label, weights))

In [ ]:
for e in edge_data:
    src = e[0]
    src_l = e[1]
    tgt = e[2]
    tgt_l = e[3]
    w = e[4]
    
    net.add_node(src, src, title=src_l)
    net.add_node(tgt, tgt, title=tgt_l)
    net.add_edge(src, tgt, value=w)

neighbor_map = net.get_adj_list()

In [ ]:
from IPython.core.display import display, HTML
# net.set_options('''
# var options = {
#     "nodes": {
#     "borderWidth": 2,
#     "borderWidthSelected": 4
#   },
#   "edges":{
#     "width":24
#   },
#   "physics": {
#     "barnesHut": {
#       "gravitationalConstant":-2000,
#       "centralGravity": 0,
#       "springLength": 60,
#       "springConstant": 0.545,
#       "damping": 0.1,
#       "avoidOverlap": 0.52
#     },
#     "maxVelocity:":50,
#     "minVelocity": 0.75,
#     "timestep": 0.5
#   }
# }
# ''')

# for atomo in range(14): 
#     net.add_node(
#         atomo,
#         label=ids[atomo],
#         x=int(100*xs[atomo]),
#         y=int(100*ys[atomo]),
#         physics=True,
#         size=30)
    


In [ ]:
# add neighbor data to node hover data
# for node in net.nodes:
#     node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
#     node["value"] = len(neighbor_map[node["id"]])

net.show("AIS_communities.html")

In [ ]:
def make_nodes_input(
    frame, 
    node_column='index', 
    weight_column=None,
    int_only=False
):
    if int_only == True:
        [i for i in geo[[node_column]].unique()[node_column]]
        
    elif weight_column != None:
        print('weights not implemented')
        return [(i, None) for i in geo[[node_column]].unique()[node_column]]
    else:
        return [(i, None) for i in geo[[node_column]].unique()[node_column]]
    
def make_edge_input(
    frame, 
    edge_column_A='index_A', 
    edge_column_B='index_B', 
    weight_column=None
):
    if weight_column != None:
        print('weights not implemented')
        
    edges = (frame.lazy()
             .select(pl.col([edge_column_A,edge_column_B]))
             .unique()
             .select(
                 pl.concat_list(
                     pl.col([edge_column_A,edge_column_B]))
                 .alias('edge'))
            ).collect().to_numpy()
    
    return [(edge[0][0], edge[0][1], None) for edge in edges]

# for weighted edges..... use one of the aggregations instead of None in the 3rd place, or combination
edges = make_edge_input(geo, edge_column_A='MMSI', edge_column_B='H3_index') # (node1, node2, payload)
try:
    edge_indicies = G.add_edges_from(edges)
except:
    print('Failed to add all edges - adding individually...')
    failures = []
    for edge in edges:
        try:
            G.add_edge(edge)
        except:
            failures.append(edge)
    print('Successfully added:', len(edges)-len(failures), '\nFailed:', len(failures))
# edges

In [ ]:
# Community Discovery algorithm(s) selection and configuration

# Clustering(s) evaluation (Fitness functions)

# Clustering(s) evaluation (Comparisons)

# Community/Statistics Visualization

# RustworkX
https://www.rustworkx.org/tutorial/introduction.html

In [ ]:
# rustworx for network creation: nodes=MMSI, edges=H3
rx_G = rx.PyGraph()


# rx_G.add_edge

# add nodes
# h3_nodes = make_nodes_input(geo, node_column='H3_index', int_only=False)
# mmsi_nodes = make_nodes_input(geo, node_column='MMSI', int_only=False)

# or add them directly and save the computation
# M_node_indicies = G.add_nodes_from(mmsi_nodes)
# H_node_indicies = G.add_nodes_from(h3_nodes)
# node_indicies = G.add_nodes_from(h3_nodes+mmsi_nodes)

In [ ]:
# set([e[0] for e in edges if type[0] != float])
# # set([e[1] for e in edges])

In [ ]:
# G.add_edge(edges[0])
# edges[0]

In [ ]:
# G.num_nodes

# Communities of vessels with intersections in space and time | ie. meetings
### done for a poxy of communications relationship results we need MMSI<->MMSI

In [ ]:
# since we have no comms link, we will use colocation in an h3 cell on the same day as a proxy
# - this does not represent a true proxy, but the structure will be right and it could be considered a meeting? so sort of
com = (
    df.lazy()
    .with_columns(pl.col('BaseDateTime').cast(pl.Date).cast(pl.Utf8).alias('Date'))
    .groupby(
        pl.col(['H3', 'Date']))
    .agg(pl.all())
    .with_row_count(name='edge_index')
    .explode(pl.exclude(['H3','Date', 'edge_index']))
).collect()
com.filter(pl.col('edge_index')==0)

In [ ]:
import networkx as nx
from cdlib import algorithms

G = nx.MultiGraph()

algorithms.louvain(G, weight=None, resolution=1, randomize=False)

In [ ]:
# edges = (df#.lazy()
#          .select(pl.col(['MMSI','H3']))
#          .select(
#              pl.concat_str(
#                  pl.col(['MMSI','H3']))
#              .alias('edge'))
#          .groupby(
#              pl.col('edge'))
#          .agg(pl.col('edge').count().alias('edge_count'))
#         )#.collect()

edges = (
    df.lazy()
    # isolate edges as an edge label to get aggregations for a payload if desired
    .with_columns(
        # combine the edge as a string initially, syntax is simpler and query optimization will eliminate the excess in the end
        pl.concat_str(pl.col(['MMSI','H3'])).alias('edge_label'),
        pl.col('H3').apply(lambda x: hash(x) + sys.maxsize + 1).alias('H3_index'),
    
        # if we want to get the number of days an edge was seen, best to change dtype before the aggregation
        pl.col('BaseDateTime').cast(str).str.slice(0,10).alias('date')) 
    # get the edge counts as to use as edge weight - use whatever aggregation you like, there are a couple examples
    .groupby('edge_label')
    .agg(
        pl.col('MMSI').unique().first().cast(pl.Int64),
        pl.col('H3').unique().first(),
        pl.col('H3_index').unique().first(),
        pl.col('edge_label').count().alias('edge_count'),
        pl.col('date').n_unique().alias('days_edge_seen'),
        pl.col('VoyageID').n_unique().alias('total_voyages'),
    )
    # .with_columns(pl.col('edge')) # may be a more elegent way, but with the query optimization we dont need to do it
).collect()
edges

In [ ]:

def get_directional_arcs(
    num_arcs, 
    return_as='list'
):
    """
    get_directional_arcs finds the left and right degree extents for the arc of a cardinal direction,
        or... with inputs greater than 4, sub-ranges for things like SW
    """
    cardinals = ['N', 'E', 'S', 'W']
    
    if num_arcs%4 != 0:
        print('num_arcs must be a multiple of 4, for now....')
        return None
        
    # get the span of the arc
    arc_length  = 360/num_arcs 

    card_arcs = []
    card_dict = {}
    
    for i in range(0,num_arcs):
        if i == 0:
            card_arcs.append([360-(arc_length/2), 0+arc_length/2])
        elif i == 1:
            card_arcs.append((card_arcs[i-1][0]+arc_length-360, card_arcs[i-1][1]+arc_length))
        else:
            card_arcs.append((card_arcs[i-1][0]+arc_length, card_arcs[i-1][1]+arc_length))
            
    return card_arcs

card_directions = get_directional_arcs(8)

# use cardinal directions....

# card = (
#     edges.lazy()
#     .with_columns(
#         pl.when(pl.col('Heading'))
#         pl.col('Heading')
#     )
# )

In [ ]:
# rustworkx takes tuples.....

e = [tuple(i) for i in edges[[
    'MMSI','H3_index', 
    'edge_count'
]].to_numpy()]
e[:2]

In [ ]:
# make a graph

G = rx.PyGraph()
G.add_edges_from(e)

In [ ]:
# Static Community Discovery

""" CDlib - Community Detection Library
https://cdlib.readthedocs.io/en/latest/tutorial.html
https://cdlib.readthedocs.io/en/latest/reference/reference.html
https://cdlib.readthedocs.io/en/latest/reference/cd_algorithms/node_clustering.html
"""
# import cdlib
from cdlib import algorithms, evaluation
import networkx as nx
import rustworkx as rx








In [ ]:
''' Community Objects
NodeClustering: 
    Node communities (either crisp partitions or overlapping groups);
FuzzyNodeClustering: 
    Overlapping node communities with explicit node to community belonging score
BiNodeClustering: 
    Clustering of a Bipartite graphs (with the explicit representation of class homogeneous communities);
AttrNodeClustering: 
    Clustering of feature-rich (node-attributed) graphs;

EdgeClustering: 
    Edge communities;
TemporalClustering: 
    Clustering of Temporal Networks;
'''
""" Community Discovery
Algorithms - and their parameters
    https://cdlib.readthedocs.io/en/latest/reference/cd_algorithms/algorithms.html

Network Complexity Notation
    n: number of nodes
    m: number of edges
    k: number of iterations
    c: number of communities
    d: average node degree

Network parameters
    Directed
    Weighted
    Bipartite
    Feature Rich
    Temporal
    
Communities
    Crisp
    Overlaps
    Nested
    Fuzzy
    Hierachical
    

"""



In [ ]:
"""
https://pyvis.readthedocs.io/en/latest/tutorial.html

"""

In [ ]:
# additional library
import communities

# References

In [ ]:
"""

cdlib - community discovery workflow

https://towardsdatascience.com/graphs-with-python-overview-and-best-libraries-a92aa485c2f8

cugraph 300x faster than networkx - but needs GPU
    https://medium.com/rapids-ai/rapids-cugraph-1ab2d9a39ec6#:~:text=RAPIDS%20cuDF%20%2B%20cuGraph%20is%20300x,is%203400x%20faster%20than%20NetworkX.

rustworkx 6 - 120x faster than networkx depending on operation - CPU
    https://qiskit.org/ecosystem/rustworkx/networkx.html
    
polars query optimization makes its speed rival cudf
    

"""

In [ ]:
142/6
14.1/.11
136/6.1
320/49